In [ ]:
from azure.ai.ml import MLClient, command, Input
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Environment, AmlCompute

In [ ]:
subscription_id = "SUBSCRIPTION_ID"
resource_group = "RESOURCE_GROUP"
workspace = "AML_WORKSPACE_NAME"

ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id,
    resource_group,
    workspace,
)

In [ ]:
cpu_compute_target = "cpu-cluster"

try:
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"{cpu_compute_target}と同一名称のクラスタが存在するので既存クラスタを使用します"
    )

except Exception:
    print("新しいコンピューティングクラスタを作成します")

    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        type="amlcompute",
        size="Standard_D2s_v3",
        min_instances=0,
        max_instances=3,
        idle_time_before_scale_down=180,
        tier="Dedicated",
    )
    print(
        f"クラスタ名 {cpu_cluster.name} 、VMサイズ {cpu_cluster.size} でクラスタ作成します"
    )

    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)

In [ ]:
custom_env_name = "aml-lightgbm"

job_env = Environment(
    name=custom_env_name,
    description="クレジットカードのデフォルトを推論するモデルを学習するためのカスタム環境",
    conda_file="senvironment.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
job_env = ml_client.environments.create_or_update(job_env)

print(
    f"{job_env.name} 環境をワークスペースへ登録しました。環境バージョンは {job_env.version} です。"
)

In [ ]:
job = command(
    inputs=dict(mode='remote'),
    code="./src/",
    command="python chapter5_job.py --mode ${{inputs.mode}}",
    environment="aml-lightgbm@latest",
    compute=cpu_compute_target,
    experiment_name="chapter5-lightgbm-job",
)

In [ ]:
job = ml_client.create_or_update(job)
ml_client.jobs.stream(job.name)